In [1]:
import undetected_chromedriver as uc 
import os
import pandas as pd
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.by import By

In [5]:
""" Test avec Seloger"""
pages = []


## Set Chrome Options
options = uc.ChromeOptions()
#options.headless=True
#options.add_argument('--headless')
#options.add_argument(f'--proxy-server={PROXY}')
#options.add_argument(f"user-agent={my_user_agent}")

## Create Undetected Chromedriver with Proxy
driver = uc.Chrome(options=options)


url = "https://www.seloger.com/list.htm?projects=1&types=2%2C1&places=%5B%7B%22inseeCodes%22%3A%5B750056%5D%7D%5D&mandatorycommodities=0&privateseller=1&enterprise=0&qsVersion=1.0&LISTING-LISTpg=1"
## Send Request
driver.get(url)
annonces = driver.find_elements(By.XPATH, '//a[@data-testid="sl.explore.coveringLink"]')
liste_liens = []
# Récupérez les liens des annonces
for lien in annonces:
    lien_annonce = lien.get_attribute('href')
    liste_liens.append(lien_annonce)
    print(lien_annonce)

driver.quit()



https://www.seloger.com/annonces/locations/appartement/paris-1er-75/seine-et-berges/162852333.htm?projects=1&types=2,1&places=[{%22inseeCodes%22:[750056]}]&mandatorycommodities=0&privateseller=1&enterprise=0&qsVersion=1.0&m=search_to_detail&lv=S
https://www.seloger.com/annonces/locations/appartement/paris-8eme-75/parc-monceau/166978385.htm?projects=1&types=2,1&places=[{%22inseeCodes%22:[750056]}]&mandatorycommodities=0&privateseller=1&enterprise=0&qsVersion=1.0&m=search_to_detail&lv=S
https://www.seloger.com/annonces/locations/appartement/paris-18eme-75/montmartre/167290247.htm?projects=1&types=2,1&places=[{%22inseeCodes%22:[750056]}]&mandatorycommodities=0&privateseller=1&enterprise=0&qsVersion=1.0&m=search_to_detail&lv=S
https://www.seloger.com/annonces/locations/appartement/paris-16eme-75/auteuil-sud/166959135.htm?projects=1&types=2,1&places=[{%22inseeCodes%22:[750056]}]&mandatorycommodities=0&privateseller=1&enterprise=0&qsVersion=1.0&m=search_to_detail&lv=S
https://www.seloger.com

In [14]:
type(liste_liens[0])

str

In [44]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

url_annonce = liste_liens[0]
results = pd.DataFrame()

l=[]
options = uc.ChromeOptions()
driver = uc.Chrome(options=options)
driver.get(url_annonce)

#Titre de l'annonce:
#titres_annonces = driver.find_elements(By.XPATH, '//div[@class="Summarystyled__Title-sc-1u9xobv-4 dbveQQ"]')[0].text
#l.append(titres_annonces)

#Prix:
#prix = driver.find_elements(By.XPATH, '//span[@class="global-styles__TextNoWrap-sc-1gbe8ip-6 gxurQr"]')[1].text #à voir si c'est toujours la 2e span 
#l.append(prix)

#Autres infos:
#Ordre: Nb pièces, (Nb chambres), Surface, Nb etages, Balcon, Terasse, Jardin, Garage, Ascenceur, Piscine  ?Parking?
# autres_infos_WebElement = driver.find_elements(By.XPATH, '//div[@class="Tags__TagContainer-sc-edpl7u-0 EPxew"]')
# autres_infos = [element.text for element in autres_infos_WebElement]
# l.append(autres_infos)

#Quartier:
# quartier = driver.find_elements(By.XPATH, '//span[@data-test="neighbourhood"]')[0].text
#l.append(quartier)

#Ville + code postal:
# ville_cp = driver.find_elements(By.XPATH, '//span[@class="Localizationstyled__City-sc-gdkcr2-1 bgtLnh"]')[0].text
# print(ville_cp)
# l.append(ville_cp)

#Description:
#Renvoie une liste vide, à creuser

#Caractéristiques:




driver.quit()







In [42]:
p_element_text

''